In the scripts in ExtractingSWRs I have provided an example of a pipeline to call data from the ABI and IBL then run through the LFP and associated meta data and return csvs with times of putative ripple events and various other metrics about the ripples.  But it can be useful to visualize the detectors results both to build your own and modify the code.  Visualization of the results of the detector are important to ensure there are not small bugs in your code and that teh data itself is nice.  It is not only useful to visulize events you are detecting but also artifacts that may prevent detection or produce false positives.  

In [6]:
# Importing libraries

# libraries
import os
import re
import subprocess 
import numpy as np
import pandas as pd
from scipy import io, signal
#from fitter import Fitter, get_common_distributions, get_distributions
import scipy.ndimage
from scipy.ndimage import gaussian_filter
from scipy.ndimage import gaussian_filter1d
import matplotlib.pyplot as plt
# for ripple detection
import ripple_detection
from ripple_detection.core import filter_ripple_band
from ripple_detection.simulate import simulate_time
from scipy import signal
import seaborn as sns
#import KernelRegDraft as kreg # custom module, not needed
#import 'Stienmetz2019Reanalyzed/KernelRegDraft.py' as kreg
import piso #can be difficult to install, https://piso.readthedocs.io/en/latest/
from scipy.ndimage import gaussian_filter
from scipy.ndimage import gaussian_filter1d
from scipy import stats
from tqdm import tqdm
import piso
from ripple_detection.simulate import simulate_time
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
import xarray as xr
from scipy import interpolate
from scipy.signal import firwin, lfilter
import sys
sys.path.append('/home/acampbell/NeuropixelsLFPOnRamp')
import Core # contains some custom functions we will find useful


The ExtractSWR scripts saved the channel in each probes CA1 LFP with the highest ripple band power.  It has already been reinterpolated from ~1250 Hz to 1500 Hz for use with the Frank Lab filter.  Note each probe has a slightly different sampling frequency which can be found in it's meta data.  I will show how to use the meta data to identify probes of interest but we will load the saved LFP data to make sure our detector is working properly. 

This is actually a useful exercise for me to build the pipeline.  Extracting -> filtering (which produces eventspersession.df) -> Global ripple detector (needs that df)